# 7-11 (舊版門市搜尋)

網址：https://emap.pcsc.com.tw

In [1]:
import os
top_dir = os.path.split(os.getcwd())[0]

import sys
if top_dir not in sys.path:
    sys.path.append(top_dir)

from utils import flatten

In [2]:
!pip install aiohttp xmltodict pydantic pandas
import asyncio
import aiohttp
import xmltodict
import pandas as pd

In [3]:
list_of_city = [
    {"city_id": "01", "city_name": "台北市"},
    {"city_id": "02", "city_name": "基隆市"},
    {"city_id": "03", "city_name": "新北市"},
    {"city_id": "04", "city_name": "桃園市"},
    {"city_id": "05", "city_name": "新竹市"},
    {"city_id": "06", "city_name": "新竹縣"},
    {"city_id": "07", "city_name": "苗栗縣"},
    {"city_id": "08", "city_name": "台中市"},
    {"city_id": "10", "city_name": "彰化縣"},
    {"city_id": "11", "city_name": "南投縣"},
    {"city_id": "12", "city_name": "雲林縣"},
    {"city_id": "13", "city_name": "嘉義市"},
    {"city_id": "14", "city_name": "嘉義縣"},
    {"city_id": "15", "city_name": "台南市"},
    {"city_id": "17", "city_name": "高雄市"},
    {"city_id": "19", "city_name": "屏東縣"},
    {"city_id": "20", "city_name": "宜蘭縣"},
    {"city_id": "21", "city_name": "花蓮縣"},
    {"city_id": "22", "city_name": "台東縣"},
    {"city_id": "23", "city_name": "澎湖縣"},
    {"city_id": "24", "city_name": "連江縣"},
    {"city_id": "25", "city_name": "金門縣"},
]
pd.DataFrame(list_of_city)

,city_id,city_name
0,01,台北市
1,02,基隆市
2,03,新北市
3,04,桃園市
4,05,新竹市
5,06,新竹縣
6,07,苗栗縣
7,08,台中市
8,10,彰化縣
9,11,南投縣


In [4]:
from seven_eleven.get_towns_by_city_id import get_towns_by_city_id

async def fn(city):
    async with aiohttp.ClientSession() as session:
        towns = await get_towns_by_city_id(session, city["city_id"])
        return [city | town.model_dump() for town in towns]

tasks = [fn(city) for city in list_of_city]

towns = await asyncio.gather(*tasks)

towns = list(flatten(towns))

pd.DataFrame(towns)

,city_id,city_name,town_id,town_name,longitude,latitude
0,01,台北市,01,松山區,121.577218,25.049837
1,01,台北市,02,信義區,121.567161,25.033147
2,01,台北市,03,大安區,121.534593,25.026482
3,01,台北市,04,中山區,121.533655,25.064427
4,01,台北市,05,中正區,121.518245,25.032251
...,...,...,...,...,...,...
362,25,金門縣,01,金湖鎮,118.418102,24.437605
363,25,金門縣,02,金沙鎮,118.426109,24.488835
364,25,金門縣,03,金城鎮,118.311377,24.414791
365,25,金門縣,04,金寧鄉,118.326959,24.451916


In [5]:
from seven_eleven.get_stores_by_city_and_town import get_stores_by_city_and_town

async def fn(town):
    async with aiohttp.ClientSession() as session:
        stores = await get_stores_by_city_and_town(session, city=town['city_name'], town=town['town_name'])
        if not isinstance(stores, list):
            return [town | stores.model_dump()]
        return [town | store.model_dump() for store in stores]

tasks = [fn(town) for town in towns[0:2]]

stores = await asyncio.gather(*tasks)

stores = list(flatten(stores))

pd.DataFrame(stores)

,city_id,city_name,town_id,town_name,longitude,latitude,store_id,store_name,address
0,01,台北市,01,松山區,121.548287,25.056391,170945,上弘,台北市松山區敦化北路168號B2
1,01,台北市,01,松山區,121.549433,25.050944,200376,小巨蛋,台北市松山區南京東路四段2號1樓
2,01,台北市,01,松山區,121.552737,25.048396,239721,中崙,台北市松山區八德路三段27號
3,01,台北市,01,松山區,121.552850,25.050888,216337,北體,台北市松山區北寧路66號
4,01,台北市,01,松山區,121.551158,25.048086,201302,台場,台北市松山區八德路三段20-2號
...,...,...,...,...,...,...,...,...,...
153,01,台北市,02,信義區,121.558867,25.018663,135562,麟運,台北市信義區和平東路三段461號461之1號1樓
154,01,台北市,02,信義區,121.577138,25.044791,215253,鑫吉,台北市信義區虎林街85號
155,01,台北市,02,信義區,121.566811,25.040167,197733,鑫國泰,台北市信義區忠孝東路五段68號24樓
156,01,台北市,02,信義區,121.566811,25.040167,277365,鑫國泰,台北市信義區忠孝東路五段68號24樓
